In [ ]:
%matplotlib inline
import os
import shutil
import platform
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle as rect
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import forward_run

In [ ]:
import os, sys
import flopy
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd

import pandas as pd
import cartopy.crs as ccrs 
import flopy.utils.binaryfile as bf
import contextily as ctx
import fiona
import seaborn as sns
import pyemu
from pathlib import Path
import helpers
from shutil import copytree, ignore_patterns

In [ ]:
import conda_scripts

In [ ]:
owhm2 = r"C:\GSP\sv\model\SV_mod_V2_owhm2\master"

In [ ]:
template_ws = "pestaws_zoom"

In [ ]:
dts = conda_scripts.utils.get_dates.get_dates()
ml = conda_scripts.sv_budget.load_sv_model.get_model(owhm2)

In [ ]:
_, mg = conda_scripts.arich_functions.get_flopy_model_spatial_reference('son')

In [ ]:
sr = helpers.get_sr()

In [ ]:
os.getcwd()

In [ ]:
egpath = Path(".").absolute()
# while egpath.name != 'examples':
#     os.chdir('..')
#     egpath = Path(".").absolute()

model_ws = Path(owhm2).absolute()
tmp_path = Path(template_ws).absolute()

EXE_DIR = Path("..","bin").absolute()
if "window" in platform.platform().lower():
    EXE_DIR = Path(EXE_DIR,"win")
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
    EXE_DIR = Path(EXE_DIR,"mac")
else:
    EXE_DIR = Path(EXE_DIR,"linux")
    
# basename = Path(model_ws).name
new_d = Path(tmp_path)
# if new_d.exists():
#     shutil.rmtree(new_d)
# Path(tmp_path).mkdir(exist_ok=True)
# creation functionality
# shutil.copytree(model_ws, new_d, ignore=ignore_patterns('*.pyc', 'output*','.git*'))

os.chdir(tmp_path)
print(Path.cwd())

In [ ]:
tmp_path

In [ ]:
#load the existing model and save it in a new dir and make sure it runs
import flopy
# model_ws = new_d.relative_to(tmp_path)
# ml = flopy.modflow.Modflow.load("freyberg.nam",model_ws=model_ws,verbose=False)
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace=tmp_path)
# ml.model_ws = "temp"
# ml.exe_name = "mfnwt"
# [shutil.copy2(os.path.join(EXE_DIR,f),os.path.join(ml.model_ws,f)) for f in os.listdir(EXE_DIR)]
# ml.write_input()
# pyemu.os_utils.run("mf-owhm.exe SVIGFM_GSP.nam", cwd=ml.model_ws)

## list file budget components as observations (or forecasts)

Here we will use ``flopy`` and ``pyemu`` to load each of the flux and volume budget components from the ``modflow`` list file to use as observations (or forecasts).  These are valuable pieces of information and since observations are free, why not include them?  This helper function writes two instruction files: ``<flx_filename>.ins`` and ``<vol_filename>.ins``

In [ ]:
# the flux budget output filename that will be written during each forward run
# flx_filename=os.path.join(owhm2,'output', "Budget.txt")

# # the volumne budget output filename that will be written during each forward run
# vol_filename = os.path.join(ml.model_ws,"vol.out")
# df_wb = pyemu.gw_utils.setup_mflist_budget_obs(os.path.join(ml.model_ws,ml.name+".list"))
# df_wb = pyemu.gw_utils.setup_mflist_budget_obs(flx_filename)

# Parameters

## pilot points

Here we will setup pilot points for several array-based ``modflow`` inputs using ``pyemu``

### setup pilot point locations

first specify what pilot point names we want to use for each model layer (counting from 0).  Here we will setup pilot points for ``hk``, ``sy`` and ``rech``.  The ``rech`` pilot points will be used as a single multiplier array for all stress periods to account for potential spatial bias in recharge.   

In [ ]:


prefix_dict = forward_run.get_prefix_dict_for_pilot_points()

This helper function is doing a lot of things: writing templates, pilot point files, and creating a shapefile of pilot points.  The ``every_n_cell`` arg is key: it decides how many cells to skip between pilot point locations - since we passed the ``model``, only active model cells get pilot points (using ``bas6.ibound``).  Like many things with ``flopy``, the ``SpatialReference`` is used to define pilot point ``x`` and ``y `` coordinates

In [ ]:
if os.path.exists(os.path.join(ml.model_ws,'pp2024')):
    print(f"{os.path.join(ml.model_ws,'pp2024')} already exists")
else:
    os.mkdir(os.path.join(ml.model_ws,'pp2024'))

In [ ]:
z = np.genfromtxt(os.path.join(ml.model_ws, 'model_arrays', 'zonation_3.csv'), delimiter = ' ')

zotther = z.copy()
zotther[zotther>8] = 0

zones = {i:zotther  for i in range(7) }
zones[1] = z
plt.imshow(zones[0])
plt.figure()
plt.imshow(zones[1])

In [ ]:
pp_cells = 7
pp_df = pyemu.pp_utils.setup_pilotpoints_grid(ml,
                                prefix_dict=prefix_dict,
                                              ibound = zones,
                                              use_ibound_zones = False,
                                every_n_cell=pp_cells,
                                pp_dir=os.path.join(ml.model_ws,'pp2024'),
                                tpl_dir=os.path.join(ml.model_ws,'pp2024'),
                                shapename=os.path.join(ml.model_ws,'pp2024',"pp.shp"))

The ``dataframe`` return has the same info as the shapefile that was written - useful info, right?

In [ ]:
pp_df.index = pp_df.parnme
pp_df

print(pp_df.pargp.unique())
pp_df

# assign values from previous model

In [ ]:
def load(folder, var, lay,name):
    arr = np.genfromtxt(os.path.join(folder, f"PARAM_{var}_G1_L{lay}.txt"), skip_header=1)

    x,y = np.indices(arr.shape, dtype = int)
    x = x.reshape((-1,1))
    y = y.reshape((-1,1))
    arr = arr.reshape((-1,1))
    arr = pd.DataFrame(data = np.hstack([x,y,arr]), columns = ['i','j', f"{name}{lay}"])
    arr = arr.set_index(['i','j'])
    return arr


ar = [load(os.path.join(owhm2, 'output'), 'HKR', layer, 'hk') for layer in np.arange(1,7)]
hk = pd.concat(ar, axis = 1,)
ar = [load(os.path.join(owhm2, 'output'), 'VKA', layer, 'vk') for layer in np.arange(1,7)]
vk = pd.concat(ar, axis = 1,)
ar = [load(os.path.join(owhm2, 'output'), 'SS', layer, 'ss') for layer in np.arange(1,7)]
ss = pd.concat(ar, axis = 1,)
sy = [load(os.path.join(owhm2, 'output'), 'Sy', layer, 'sy') for layer in np.arange(1,2)][0]

aq = pd.concat([hk, vk, ss, sy], axis = 1)
aq

In [ ]:
cols = pp_df.columns

pp_ren = pd.merge(pp_df, aq, on = ['i', 'j'], how = 'left')

for prop in pp_df.pargp.unique():
    if prop in aq.columns:
        c = pp_ren.pargp== prop
        pp_ren.loc[c, 'parval1'] = pp_ren.loc[c, prop]
    else:
        print(f"skipping {prop} bc it's not in loaded arrays")

# set vk to 0.1 for layer 1 and 0.001 for others
pp_ren.loc[pp_ren.pargp.str.startswith('vk'), 'parval1'] = 0.1
pp_ren.loc[pp_ren.pargp=='vk1', 'parval1'] = 0.1


pp_ren = pp_ren.loc[:,cols]



# set bounds to values

In [ ]:
import re

def remove_numbers(input_string):
    return re.sub(r'\d+', '', input_string)

pp_ren.pargp.apply(remove_numbers).unique()

In [ ]:
bounds = dict(hk = [0.0001, 500],
sy = [0.01, 0.3],
vk = [0.001, .1],
ss = [1E-7, 1E-4],
fmp_vk = [0.0001, 10],
drn_k= [0.0001, 10])
bounds

In [ ]:
for n, group in pp_ren.groupby(pp_ren.pargp.apply(remove_numbers)):
    pp_ren.loc[group.index,['parlbnd','parubnd']] = bounds[n]

In [ ]:
pp_ren.to_csv('pilot_point_info.csv')

### geostats and kriging
now that we have pilot points setup, we need to solve the kriging equations for each model cell using pilot point locations.  Since we only have a single set of pilot points that we are reusing for several array-based ``modflow`` inputs, we only need to get the kriging factors once

In [ ]:
hk_pp = pyemu.pp_utils.pp_file_to_dataframe(os.path.join(ml.model_ws,'pp2024',"hk1pp.dat"))
hk_pp2 = pyemu.pp_utils.pp_file_to_dataframe(os.path.join(ml.model_ws,'pp2024',"hk2pp.dat"))

In [ ]:
hk_pp.head()

In [ ]:
hk_pp2.head()

Let's setup a geostatistical structure.  The contribution doesn't matter for pilot point interpolation, but it does matter when we want to form a prior parameter covariance matrix - we will get to that later.  A good rule of thumb is to use an ``a`` value that is three times the pilot point spacing.  Also, since the all of these pilot points will be log transformed, we need to use a log-based geostatistical structure


In [ ]:
a = pp_cells * ml.dis.delr.array[0] * 3.0
v = pyemu.geostats.ExpVario(contribution=1.0,a=a)
gs = pyemu.geostats.GeoStruct(variograms=v,transform="log")
gs.plot()

This is where things get fun.  First we create an ``OrdinaryKrige`` object


In [ ]:
ok = pyemu.geostats.OrdinaryKrige(geostruct=gs,point_data=hk_pp)
ok2 = pyemu.geostats.OrdinaryKrige(geostruct=gs,point_data=hk_pp2)

Now we use a helper function to solve the kriging factors for each active model cell: ``OrdinaryKrige.calc_factors_grid()`` includes all the standard kriging arguments, such as search radius, min and max interpolation points,zone_array, as well as the option to save the kriging variance array 

Note: we need to pass out model's spatial reference information. For flopy this used to be contained in model.sr
      However this has been superseded by model.modelgrid. To avoid reliance on a changing (and not always backward
      compatible) code base the sr method has been abstracted into pyemu.

In [ ]:
plt.imshow(zones[1])

In [ ]:
# sr = pyemu.helpers.SpatialReference.from_namfile(os.path.join(ml.model_ws, ml.namefile),
#                                                  delr=ml.dis.delr, delc=ml.dis.delc)
ok.calc_factors_grid(sr, 
                     # zone_array=zones[0],
                     var_filename=os.path.join(ml.model_ws,'pp2024',"layer1_var.dat"))
print('\nstarting layer 2\n')
ok2.calc_factors_grid(sr, 
                     # zone_array=zones[1],
                     var_filename=os.path.join(ml.model_ws,'pp2024',"layer2_var.dat"))
print('done')

Ok, we know that this function is slow for bigly models, but it is super convienent and allows a lot of flexibility.  So, once we have calculated the kriging factors for each active model cell, we need to write this to a factors file

In [ ]:
ok.to_grid_factors_file(os.path.join(ml.model_ws,'pp2024', "pp.fac"))
ok2.to_grid_factors_file(os.path.join(ml.model_ws,'pp2024', "pp2.fac"))

# this has been added to helpers for model runs

In [ ]:
import importlib
importlib.reload(forward_run)

In [ ]:
# def write_pilot_point(layer, prop, model_ws):
#     if layer !=1:
#         factors_file = os.path.join(model_ws,'pp2024', "pp.fac")
#     else:
#         factors_file = os.path.join(model_ws,'pp2024', "pp2.fac")

#     out_file = os.path.join(model_ws, 'pp2024_out',f"{prop}.txt")
    
#     pp_file =  os.path.join(model_ws, 'pp2024',f"{prop}pp.dat")
#     assert os.path.exists(pp_file), f"pp_file does not exist {pp_file}"
#     print(f"pp_file = {pp_file}, factors_file={factors_file}, out_file={out_file}")
    
#     hk_arr = pyemu.geostats.fac2real(pp_file, factors_file=factors_file, out_file=out_file)

# def write_all_pp(model_ws):
#     prefix_dict= {0:["hk1","sy1","vk1"],
#                  1:["hk2","ss2","vk2",'fmp_vk', 'drn_k'],
#                  2:["hk3","ss3","vk3"],
#                  3:["hk4","ss4","vk4"],
#                  4:["hk5","ss5","vk5"],
#                  5:["hk6","ss6","vk6"]}
#     for lay in prefix_dict.keys():
#         for par in prefix_dict[lay]:
#             write_pilot_point(lay, par, model_ws)

forward_run.write_all_pp(ml.model_ws)


# pp_file = 'hkpp1.dat'

# out_folder = ''
# out_file='freyberg6.npf_k_layer1.txt'

# pp_file = 'rchpp.dat'
# hk_arr = pyemu.geostats.fac2real(pp_file, factors_file=pp_file+'.fac',out_file='rch0_fac.txt')

Let's check out that kriging variance array....


In [ ]:
var_arr = np.ma.masked_invalid(np.loadtxt(os.path.join(ml.model_ws,'pp2024',"layer1_var.dat")))
fig = plt.figure(figsize=(8.5,11))
mm = conda_scripts.make_map.make_map('layer1')
ax = mm.plotloc(fig, locname = 'SON_MOD')
ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,var_arr,alpha=0.5)
ax.scatter(hk_pp.x, hk_pp.y,marker='o', c = hk_pp.zone,s=25, cmap = 'tab10', ec = 'k')
plt.savefig('pilot_points_lay1.png')

In [ ]:
hk_pp2

In [ ]:
var_arr = np.ma.masked_invalid(np.loadtxt(os.path.join(ml.model_ws,'pp2024',"layer2_var.dat")))
fig = plt.figure(figsize=(8.5,11))
mm = conda_scripts.make_map.make_map('layer2')
ax = mm.plotloc(fig, locname = 'SON_MOD')
ax.pcolormesh(sr.xcentergrid,sr.ycentergrid,var_arr,alpha=0.5)
ax.scatter(hk_pp2.x, hk_pp2.y,marker='o', c = hk_pp2.zone,s=25, cmap = 'tab10', ec = 'k')
plt.savefig('pilot_points_lay2.png')

In [ ]:
os.getcwd()

# add zone multipliers

In [ ]:
mult_path = os.path.join('zone_pest_mult')

In [ ]:
if not os.path.exists(mult_path):
    os.makedirs(mult_path, exist_ok=True)

In [ ]:
zdict = forward_run.get_prefix_dict_for_pilot_points()

In [ ]:
z = np.genfromtxt(os.path.join(ml.model_ws, 'model_arrays', 'zonation_3.csv'), delimiter = ' ')

zotther = z.copy()
zotther[zotther>8] = 0

zones = {i:zotther  for i in range(7) }
zones[1] = z
plt.imshow(zones[1])

z = conda_scripts.arich_functions.array2rc(zones[1],'zone').astype({'zone':int})

aliases = {1: 'Bay', 2: 'EastSide', 3: 'SouthCent', 4: 'Kenwood', 5: 'VOM', 6: 'AguaCal',7:'WestSide',8:'CitySon',9:'Highlands'}
z.loc[:,'zone'] =z.loc[:,'zone'].replace(aliases)
z = z.query("zone!=0")

geoms = conda_scripts.arich_functions.get_model_shp(ml.modelgrid, 2226).drop(columns = ['row','col'])

z = gpd.GeoDataFrame(pd.merge(z, geoms, on = ['i','j']))
zplot = z.dissolve('zone').reset_index()

In [ ]:
zz = [z for _,z in zones.items()]
zz = np.stack(zz)
zz = zz[:6,:,:]
zz[ml.bas6.ibound.array==0] = np.nan
zz = np.array(zz, dtype = int)
zz.shape

In [ ]:
fig, ax = plt.subplots(2,3)
ax = ax.ravel()

for i in range(6):

    ax[i].imshow(zz[i])
    ax[i].set_title(i+1)

In [ ]:
prefix = forward_run.get_prefix_dict_for_pilot_points()
prefix

# write array files while adding parameter locations for paremeter values


In [ ]:

for lay, vals in prefix.items():
    for val in vals:
        # string_array = np.zeros(z.shape, dtype ='U20')
        string_array = np.zeros(zz[i].shape, dtype ='U30')
        string_array[:,:] = '1e30'
        for zonenum, zonename in aliases.items():
            
            c_ = zz[i]==zonenum
            fff = "zonemult_{:}_{:}".format(val, zonename)
            string_array[c_] = f"~{fff:20s}~"
            print(zonenum, val, zonename)
            # print(np.unique(base.ravel()))
            # # Use np.vectorize to apply the dictionary mapping
            # string_array = np.vectorize(lambda x: '' if x is None else aliases.get(x))(base)
        file = os.path.join(mult_path, f'zonemult_{val}.csv.tpl')
        
        with open(file,'w') as wrt:
            wrt.write('ptf ~\n')
        with open(file,'a') as wrt:
            np.savetxt(wrt, string_array, fmt = '%s', delimiter=',')


# create layer multiplier

In [ ]:

for lay, vals in prefix.items():
    for val in vals:       
        
        string_array = np.zeros(zz[i].shape, dtype ='U30')
        string_array[:,:] = '1e30'

        c_ = zz[i]>0
        
        fff = "laymult_{:}".format(val)
        string_array[c_] = f"~{fff:20s}~"

        file = os.path.join(mult_path, f'laymult_{val}.csv.tpl')
        
        with open(file,'w') as wrt:
            wrt.write('ptf ~\n')
        with open(file,'a') as wrt:
            np.savetxt(wrt, string_array, fmt = '%s', delimiter=',')